<a href="https://colab.research.google.com/github/Kostratana/logistic_optimization_genetic_ants_algorithms/blob/main/%22Clustering_by_cars%2C_map_by_longitude_and_latitude%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import folium
from sklearn.cluster import KMeans
import requests

Загружаем файлы

In [ ]:
url = 'https://docs.google.com/spreadsheets/d/1M3ON3dw8sUJx9WPV14q0FRz8w4d7uZQw/export?format=xlsx'
r = requests.get(url, allow_redirects=True)
route_path = 'Рейсы17_22042023.xlsx'
open(route_path, 'wb').write(r.content)

url = 'https://docs.google.com/spreadsheets/d/1n3AnbiELQc7NJ6lCWiomz1PXYL8hjRiD/export?format=xlsx'
r = requests.get(url, allow_redirects=True)
car_path = 'автомобили.xlsx'
open(car_path, 'wb').write(r.content)


df_route = pd.read_excel(route_path)
df_trucks = pd.read_excel(car_path)

ride_names = ['date', 'flight', 'vehicle', 'weight', 'volume', 'palletes', 'driver',
            'document', 'doc_weight', 'doc_vol', 'doc_palletes',
              'lon', 'lat', 'Load_Order']
truck_names = ['truck','truck_weight','truck_volume','truck_palletes']

df_route.columns = ride_names
df_trucks.columns = truck_names

In [ ]:
df_route.head()

,date,flight,vehicle,weight,volume,palletes,driver,document,doc_weight,doc_vol,doc_palletes,lon,lat,Load_Order
0,2023-04-17,10910622,58,5000,27.0,12,90021,10919764,363.28,0.539532,0.58,61.405797,55.263773,1
1,2023-04-17,10910622,58,5000,27.0,12,90021,10913393,263.52,0.358008,0.37,61.374230,55.148575,2
2,2023-04-17,10910622,58,5000,27.0,12,90021,10913360,252.24,0.341880,0.36,61.401736,55.171504,3
3,2023-04-17,10910622,58,5000,27.0,12,90021,10913368,259.44,0.357588,0.36,61.360503,55.189028,4
4,2023-04-17,10910622,58,5000,27.0,12,90021,10906288,10.00,0.050000,0.00,61.610442,55.116516,5


In [ ]:
df_trucks.head()

,truck,truck_weight,truck_volume,truck_palletes
0,34,12600,40,15
1,51,1400,10,4
2,39,3800,24,10
3,42,3800,24,10
4,38,3800,24,10


DataFrame по заданной дате

In [ ]:
def create_orders(df_route, specific_date):
    """
    список для заданной даты.

    :param df_route: DataFrame .
    :param specific_date: дата.
    :return: словарь.
    """
    df_filtered = df_route[df_route['date'] == specific_date]

    orders = []
    for _, row in df_filtered.iterrows():
        order = {
            'id': row['document'],
            'weight': row['doc_weight'],
            'volume': row['doc_vol'],
            'pallets': row['doc_palletes'],
            'longitude': row['lon'],
            'latitude': row['lat']
        }
        orders.append(order)

    return orders

def create_vehicles(df_trucks):
    """
    Создает список транспортных средств из DataFrame.

    :param df_truck: информацией по транспорту.
    :return: словарь .
    """
    vehicles = []
    for _, row in df_trucks.iterrows():
        vehicle = {
            'id': row['truck'],
            'capacity': row['truck_weight'],
            'volume': row['truck_volume'],
            'max_pallets': row['truck_palletes']
        }
        vehicles.append(vehicle)

    return vehicles


Генерируем кластеры

In [ ]:
def generate_clusters(orders, num_clusters):
    """
    Генерирует кластеры .

    :param orders: Список заказов с гео координаторами.
    :param num_clusters: Название кластеров.
    :return: словарь.
    """
    # Извлечение координат заказов
    coordinates = np.array([(order['longitude'], order['latitude']) for order in orders])

    # Применение K-средних для кластеризации
    kmeans = KMeans(n_clusters=num_clusters, n_init=10, random_state=0).fit(coordinates)
    clusters = kmeans.labels_

    # Группировка заказов по кластерам
    clustered_orders = {}
    for cluster_id in range(num_clusters):
        clustered_orders[cluster_id] = [order for order, cluster in zip(orders, clusters) if cluster == cluster_id]
        print('кластер', cluster_id, ' : ', clustered_orders[cluster_id])


    return clustered_orders

Создаем кластры для карты

In [ ]:
def plot_clusters_on_map(clustered_orders,bounding_box):
    center_lat = (bounding_box[0] + bounding_box[1]) / 2
    center_lon = (bounding_box[2] + bounding_box[3]) / 2


    m = folium.Map(location=[center_lat, center_lon], zoom_start=5)

      # bounding box
    folium.Rectangle(
        bounds=[(bounding_box[0], bounding_box[2]), (bounding_box[1], bounding_box[3])],
        color="#ff7800",
        fill=True,
        fill_color="#ff7800",
        fill_opacity=0.2,
    ).add_to(m)

    colors = ['red', 'blue', 'green', 'purple', 'orange', 'darkred', 'lightred', 'beige', 'darkblue', 'darkgreen', 'cadetblue',
              'darkpurple', 'white', 'pink', 'lightblue', 'lightgreen', 'gray', 'black', 'lightgray']

    for cluster_id, orders in clustered_orders.items():
        cluster_color = colors[cluster_id % len(colors)]
        for order in orders:

            popup_text = f"Координаты: {order['latitude']}, {order['longitude']}"
            folium.CircleMarker(
                location=[order['latitude'], order['longitude']],
                radius=5,
                color=cluster_color,
                fill=True,
                fill_color=cluster_color,
                popup=popup_text
            ).add_to(m)

    return m

Распределение Кластеров По Машинам

In [ ]:
def distribute_clusters_to_vehicles(clustered_orders, vehicles):
    vehicle_distribution = {vehicle['id']: [] for vehicle in vehicles}
    vehicle_load = {vehicle['id']: {'current_weight': 0, 'current_volume': 0, 'current_pallets': 0} for vehicle in vehicles}
    work_vehicles = set() # рабочие машины

    for cluster_id, orders in clustered_orders.items():
        # сортируем машины
        sorted_orders = sorted(orders, key=lambda x: x['weight'], reverse=True)
        print('загрузка кластера ', cluster_id)

        for vehicle in vehicles:
            if vehicle['id'] in work_vehicles:
                continue

            for order in sorted_orders:
                if can_accommodate_order(vehicle, order, vehicle_load):
                    vehicle_distribution[vehicle['id']].append(order)
                    vehicle_load[vehicle['id']]['current_weight'] += order['weight']
                    vehicle_load[vehicle['id']]['current_volume'] += order['volume']
                    vehicle_load[vehicle['id']]['current_pallets'] += order['pallets']
                    sorted_orders.remove(order)  # Удаляем заказ из списка, так как он размещен
                    #print(f"Заказ ID {order['id']} размещен в автомобиле {vehicle['id']}.")
                    #print(f"Текущая загрузка автомобиля {vehicle['id']}: Вес {vehicle_load[vehicle['id']]['current_weight']}, Объем {vehicle_load[vehicle['id']]['current_volume']}, Паллеты {vehicle_load[vehicle['id']]['current_pallets']}")
            if vehicle_distribution[vehicle['id']]:
                work_vehicles.add(vehicle['id'])  # Транспортное средство используется для этого кластера

        if not sorted_orders:
            continue  # Все заказы кластера размещены, переходим к следующему кластеру


        if sorted_orders:
         print("Некоторые заказы не удалось разместить:")
         for order in sorted_orders:
            print(f"Заказ ID: {order['id']}, Вес: {order['weight']}, Объем: {order['volume']}, Паллеты: {order['pallets']}")

    return vehicle_distribution


def can_accommodate_order(vehicle, order, vehicle_load):
    """
    Проверяет, может ли транспортное средство вместить заказ, учитывая текущую загрузку.
    """
    new_weight = vehicle_load[vehicle['id']]['current_weight'] + order['weight']
    new_volume = vehicle_load[vehicle['id']]['current_volume'] + order['volume']
    new_pallets = vehicle_load[vehicle['id']]['current_pallets'] + order['pallets']

    can_accommodate = (new_weight <= vehicle['capacity'] and
                       new_volume <= vehicle['volume'] and
                       new_pallets <= vehicle['max_pallets'])

    return can_accommodate

Координаты по точкам

In [ ]:
def get_bounding_box(points):
    lats = [point['latitude'] for point in points]
    lngs = [point['longitude'] for point in points]
    return min(lats), max(lats), min(lngs), max(lngs)

In [ ]:
df_route = df_route.dropna(subset=['lon', 'lat'])

specific_date = '2023-04-17'

orders = create_orders(df_route, specific_date)

total_weight = sum(order['weight'] for order in orders)
total_volume = sum(order['volume'] for order in orders)
total_pallets = sum(order['pallets'] for order in orders)
print("\nОбщий вес: ", total_weight)
print("Общий объем: ", total_volume)
print("Общее количество паллет: ", total_pallets)

vehicles = create_vehicles(df_trucks)

num_clusters = 3 # количество кластеров

clustered_orders = generate_clusters(orders, num_clusters)
vehicle_distribution = distribute_clusters_to_vehicles(clustered_orders, vehicles)


Общий вес:  31194.309999999994
Общий объем:  93.37504999999999
Общее количество паллет:  90.25000000000003
кластер 0  :  [{'id': 10919764, 'weight': 363.28, 'volume': 0.539532, 'pallets': 0.58, 'longitude': 61.405797, 'latitude': 55.263773}, {'id': 10913393, 'weight': 263.52, 'volume': 0.358008, 'pallets': 0.37, 'longitude': 61.37423, 'latitude': 55.148575}, {'id': 10913360, 'weight': 252.24, 'volume': 0.34188, 'pallets': 0.36, 'longitude': 61.401736, 'latitude': 55.171504}, {'id': 10913368, 'weight': 259.44, 'volume': 0.357588, 'pallets': 0.36, 'longitude': 61.360503, 'latitude': 55.189028}, {'id': 10906288, 'weight': 10.0, 'volume': 0.05, 'pallets': 0.0, 'longitude': 61.610442, 'latitude': 55.116516}, {'id': 10906238, 'weight': 10.0, 'volume': 0.05, 'pallets': 0.0, 'longitude': 61.619146, 'latitude': 55.117206}, {'id': 10904986, 'weight': 10.0, 'volume': 0.05, 'pallets': 0.0, 'longitude': 61.415534, 'latitude': 54.888801}, {'id': 10905022, 'weight': 10.0, 'volume': 0.05, 'pallets': 

Визуализация кластеров на карте по долготе 50 и широте 60

In [ ]:
base_latitude = 50
base_longitude = 60
base_point = {'latitude': base_latitude, 'longitude': base_longitude}

all_points = [base_point]  # Включаем базу в список точек
# Добавляем точки доставки всех заказов из всех кластеров
for orders in vehicle_distribution.values():
    all_points.extend([{'latitude': order['latitude'], 'longitude': order['longitude']} for order in orders])

# Определяем ограничивающий прямоугольник для всех точек
bounding_box = get_bounding_box(all_points)

m = plot_clusters_on_map(clustered_orders, bounding_box)
m